In [ ]:
import hashlib, math, os, subprocess
from multiprocessing import Process
import xlearn as xl
import numpy as np
import pandas as pd
from pandas import DataFrame as DF

In [ ]:
def df_tolist(x):
    start=1
    end=51
    l=end-start
    temp=[0]*(l)
    for i in range(0,l):
        temp[i]=x[i+start]
    return temp

In [ ]:
class FfmEncoder():
    def __init__(self, field_names, label_name, nthread=1):
        self.field_names = field_names
        self.nthread = nthread
        self.label = label_name

    def gen_feats(self, row):
        feats = []
        for field in self.field_names:
            value = row[field]
#             key = field + '-' + str(value)
            key=str(value)
            feats.append(key)
        return feats

    def gen_fm_feats(self, feats):
        feats = ['{0}:{1}:1'.format(field, feat) for (field, feat) in feats]
        return feats

    def convert(self, df, path, i):
        lines_per_thread = math.ceil(float(df.shape[0]) / self.nthread)
        sub_df = df.iloc[i * lines_per_thread: (i + 1) * lines_per_thread]
        tmp_path = path + '_tmp_{0}'.format(i)
        with open(tmp_path, 'w') as f:
            for index,row in sub_df.iterrows():
                feats = []
                for i, feat in enumerate(self.gen_feats(row)):
                    feats.append((i, feat))
                feats = self.gen_fm_feats(feats)
                f.write(str(int(row[self.label])) + ' ' + ' '.join(feats) + '\n')

    def parallel_convert(self, df, path):
        processes = []
        for i in range(self.nthread):
            p = Process(target=self.convert, args=(df, path, i))
            p.start()
            processes.append(p)
        for p in processes:
            p.join()

    def delete(self, path):
        for i in range(self.nthread):
            os.remove(path + '_tmp_{0}'.format(i))

    def cat(self, path):
        if os.path.exists(path):
            os.remove(path)
        for i in range(self.nthread):
            cmd = 'cat {svm}_tmp_{idx} >> {svm}'.format(svm=path, idx=i)
            p = subprocess.Popen(cmd, shell=True)
            p.communicate()

    def transform(self, df, path):
#         print('converting data......',end='\n')
        self.parallel_convert(df, path)
        self.cat(path)
        self.delete(path)

In [ ]:
data1=pd.read_csv("/home/ubuntu/wangnan/minfo_candidate_hottest3000_1.csv")
data1.drop(labels=['name','director','scriptwriter','mainPlayers','categories','productCountry','duration','releasedTime','tags','ratingCount','star5','star4','star3','star2','star1','language'], axis=1,inplace = True)
data1=data1.dropna()
data1['timeMode']=data1['0'].apply(lambda x:int(x)-2005)
data1.drop(labels=['0','timeStamp'], axis=1,inplace = True)
data1['mid_list']=data1.apply(df_tolist,axis=1)
data3=data1[['mid','timeMode','mid_list']].copy()
data3.to_csv("/data/ng/FFM/mid_tolist_1000.csv",header=True,index=False)

In [ ]:
df_m=pd.read_csv("/data/ng/FFM/mid_tolist_1000.csv")

In [ ]:
df_m.head()

In [ ]:
user_list=pd.read_csv("/home/ubuntu/siyuan/UserMovie_test2.txt",header=None,names=['uid'])
user_list=user_list['uid'].tolist()

In [ ]:
i=0
movie_num=len(df_m)
field_names =['uid', 'mid', 'timeStamp', 'timeMode', 'mid_list']
recommand_file=open("/data/ng/FFM/recommand_result.txt",'w')
for uid in user_list:
    df_u=pd.DataFrame([uid]*movie_num)
    timeStamp=pd.DataFrame(['0']*movie_num)
    star=pd.DataFrame(['0']*movie_num)
    val=pd.concat([df_u,df_m],axis=1)    
    val.insert(2,'timeStamp',timeStamp)
    val.insert(0,'star',star)
    val.rename(columns={0:'uid'},inplace=True)
    ###生成测试文件
    write_path = '/data/ng/FFM/'
    fe = FfmEncoder(field_names,label_name='star',nthread=8)
    fe.transform(val, write_path+'val_without_tag.ffm')
#     print('converting data finished',end='\n')
    ###开始进行预测
#     print('predict begin......',end='\n')
    ffm_model = xl.create_ffm()
    ffm_model.setTest("/data/ng/FFM/val_without_tag.ffm")
    ffm_model.setSigmoid()
    ffm_model.predict("/data/ng/FFM/model2.out", "/data/ng/FFM/output2.txt")
#     print('predict finished',end='\r')
    ###开始排序输出
#     print('recommand begin.....',end='\n')
    m_score=pd.read_csv("/data/ng/FFM/output2.txt",'r',header=None,names=['score'])
    m_score=pd.concat([df_m['mid'],m_score],axis=1)
    m_score=m_score.sort_values(['score'],ascending=False)
    command=m_score.iloc[0:50]
    command_list=command['mid'].tolist()
    for command_movie in command_list:
        temp=str(uid)+'\t'+str(command_movie)+'\n'
        recommand_file.write(temp)
#     print('recommand finished',end='\n')
    i+=1
    print(i,end='\r')
#     break
recommand_file.close()